## Notebook-Interface for controlling the preprcessing part

In [218]:
# source /Users/me/Desktop/Code/venvs/open_diabetis/bin/activate


# Link to github repo:


import os
import json
import datetime
import csv

import pandas as pd
import gzip


'''
%load_ext lab_black
%matplotlib inline
'''
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")


from data_io import File_IO, Database_IO
from data_wrangling import  Data_Wrangling
from data_summarization import Data_Summarization
from setup_config import Setup_Config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [210]:
setup = Setup_Config('config.ini') # loads a setup file with variables in .ini format
# the .ini file contains e.g. database connections and other settings.
#### ATTENTION: make sure to add this file to successfully to .gitignore to make sure it dont become pushed to the public repo.

file_io = File_IO() # handles input and output operations

data_wrangling = Data_Wrangling(data_io=file_io) # includes all the transformations
data_summarizations = Data_Summarization(data_io=file_io)


# if db not usage not wished, unccoment temporarly
database_io = Database_IO(
    host_ip=setup.config.db.host_ip, 
    port=setup.config.db.port, 
    db_user=setup.config.db.db_user, 
    db_pw=setup.config.db.db_pw, 
    db_name=setup.config.db.db_name) # use default postgres db


Connected to the database. List of schemes: ['information_schema', 'public']


In [ ]:
# 1. Read in all json files for each patient

parent_folder = "/Users/me/Desktop/Code/VS-Projects/charite/OpenHumansDataTools/data" # if not exists create dir before
patients_jsons_dict = file_io.get_dict_of_folder_and_json_files(parent_folder)


In [ ]:
# 2. merge all json files in one csv for each patient with columns:: path, is_valid, starttime, value, value_str

outpath = '/Users/me/Desktop/Code/VS-Projects/charite/OpenHumansDataTools/temp/data' # if not exists create dir before
data_wrangling.group_patients_csv_data(patients_jsons_dict, outpath=outpath)


# TODO: which time zone is used in uploaded data. where to find the timezone
# there is a file called UploadInfo, which provides an utc offset. 
# calculate the utc offset on the unix data and convert to python date format
# if somebody switches timezones, we have to capture this and check for each upload seperatly

# the utc time is meassured in miliseconds, as well as the utc offset


In [ ]:
# 3. use the merged files in predefined csv format calculate some statistics

folder = '/Users/me/Desktop/Code/VS-Projects/charite/OpenHumansDataTools/temp/data'

file_names = file_io.get_csv_files(folder=folder)

outpath='/Users/me/Desktop/Code/VS-Projects/charite/OpenHumansDataTools/temp/summarizations'
data_summarizations.summarize_data(csv_files=file_names, outpath=outpath)

In [224]:
# 5. insert data from csv files in new database table

# 1. get csv file paths
# 2. create new db table
# 2b optional check if columns are in all csv files the same as well as the type ...
# 3. insert data from all csv files recognized in the given file path to db


# 1. get csv file paths
file_paths = file_io.get_csv_files(folder=folder)

# choose a name for the target table, where the data will be saved (in current schema of the database specified in the config.ini file)
target_table_name = 'open_uploaded_all'

# 2. create new db table
### creating a new database table, assuming typs are the same in all frames
first_frame = file_io.read_csv_pandas(file_paths[0])
type_dict = database_io.get_df_column_types(first_frame) # dictionary of recognized columns and tyoes
database_io.create_new_db_table(table_name=target_table_name, type_dict=type_dict)

# creates indices, which enable faster filtering of the database tables, e.g. with sql commands
# TODO: If this function runs again, delete existsing index and overwrite. Currently we have to delete the db table manually before or recreate the index.
database_io.create_new_index(table_name=target_table_name, index_name='open_uploaded_all_path', index_cols=['path'] )
database_io.create_new_index(table_name=target_table_name, index_name='open_uploaded_all_patient_id', index_cols=['patient_id'] )

In [229]:
type_dict

{'index': numpy.int64,
 'path': str,
 'value': str,
 'value_str': float,
 'starttime': numpy.float64,
 'duration': numpy.float64,
 'isValid': bool,
 'patient_id': numpy.int64}

In [233]:
target_typ_dict= {
    'path': str,
    'value': float,
    'value_str': str,
    'starttime': pd._libs.tslibs.timestamps.Timestamp,
    'duration': float,
    'isValid': bool,
    'patient_id': int
}

In [234]:
def convert_dataframe_type(dataframe, target_typ_dict):
    """ Converts the columns of a pandas dataframe to the specified types.
    
    Args:
        dataframe (pd.DataFrame): The pandas dataframe to be converted.
        target_typ_dict (dict): The dictionary containing the target column type information.
    """
    for col, typ in target_typ_dict.items():
        dataframe[col] = dataframe[col].astype(typ)
        
    return dataframe

In [236]:
convert_dataframe_type(dataframe=test_frame, target_typ_dict=target_typ_dict)

ValueError: could not convert string to float: 'False'

In [ ]:
test_frame = file_io.read_csv_pandas(file_paths[0])
test_frame.head()

,path,value,value_str,starttime,duration,isValid,patient_id
0,treatments.isdeletion,False,NaN,1.623776e+12,NaN,True,84181797
1,treatments.mealbolus,False,NaN,1.623776e+12,NaN,True,84181797
2,treatments.issmb,True,NaN,1.623776e+12,NaN,True,84181797
3,treatments.insulininterfaceid,2,NaN,1.623776e+12,NaN,True,84181797
4,treatments.insulin,0.5,NaN,1.623776e+12,NaN,True,84181797


In [ ]:
# 3. insert data from all csv files recognized in the given file path to db

for file_path in file_paths:
    frame = file_io.read_csv_pandas(file_path)
    database_io.append_frame_to_sql_table(table_name=target_table_name,data=frame)

DataError: (psycopg2.errors.InvalidTextRepresentation) invalid input syntax for type double precision: "COB: 0, Dev: 2.3, BGI: -0.2, ISF: 3.0, CR: 9, Target: 6.5, minPredBG 9.7, minGuardBG 11.7, IOBpredBG 7.4, UAMpredBG 12.1; Eventual BG 12.1 >= 7.0,  insulinReq 1.07; setting 30m low temp of 0.1U/h. Microbolusing 0.5U."
LINE 1: ...L, 84181797),(752, 'apsdata.result.reason', NULL, 'COB: 0, D...
                                                             ^

[SQL: INSERT INTO open_uploaded_all (index, path, value, value_str, starttime, duration, "isValid", patient_id) VALUES (%(index)s, %(path)s, %(value)s, %(value_str)s, %(starttime)s, %(duration)s, %(isValid)s, %(patient_id)s)]
[parameters: ({'index': 0, 'path': 'treatments.isdeletion', 'value': 'False', 'value_str': None, 'starttime': 1623775884085.0, 'duration': None, 'isValid': True, 'patient_id': 84181797}, {'index': 1, 'path': 'treatments.mealbolus', 'value': 'False', 'value_str': None, 'starttime': 1623775884085.0, 'duration': No

In [ ]:
for file_path in file_paths:
    frame = file_io.read_csv_pandas(file_path)
    type_dict = database_io.get_df_column_types(first_frame)
    print(type_dict)
    

{'index': <class 'numpy.int64'>, 'path': <class 'str'>, 'value': <class 'str'>, 'value_str': <class 'float'>, 'starttime': <class 'numpy.float64'>, 'duration': <class 'numpy.float64'>, 'isValid': <class 'bool'>, 'patient_id': <class 'numpy.int64'>}
{'index': <class 'numpy.int64'>, 'path': <class 'str'>, 'value': <class 'str'>, 'value_str': <class 'float'>, 'starttime': <class 'numpy.float64'>, 'duration': <class 'numpy.float64'>, 'isValid': <class 'bool'>, 'patient_id': <class 'numpy.int64'>}


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3364: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


{'index': <class 'numpy.int64'>, 'path': <class 'str'>, 'value': <class 'str'>, 'value_str': <class 'float'>, 'starttime': <class 'numpy.float64'>, 'duration': <class 'numpy.float64'>, 'isValid': <class 'bool'>, 'patient_id': <class 'numpy.int64'>}


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3364: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


{'index': <class 'numpy.int64'>, 'path': <class 'str'>, 'value': <class 'str'>, 'value_str': <class 'float'>, 'starttime': <class 'numpy.float64'>, 'duration': <class 'numpy.float64'>, 'isValid': <class 'bool'>, 'patient_id': <class 'numpy.int64'>}


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3364: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


{'index': <class 'numpy.int64'>, 'path': <class 'str'>, 'value': <class 'str'>, 'value_str': <class 'float'>, 'starttime': <class 'numpy.float64'>, 'duration': <class 'numpy.float64'>, 'isValid': <class 'bool'>, 'patient_id': <class 'numpy.int64'>}
{'index': <class 'numpy.int64'>, 'path': <class 'str'>, 'value': <class 'str'>, 'value_str': <class 'float'>, 'starttime': <class 'numpy.float64'>, 'duration': <class 'numpy.float64'>, 'isValid': <class 'bool'>, 'patient_id': <class 'numpy.int64'>}
{'index': <class 'numpy.int64'>, 'path': <class 'str'>, 'value': <class 'str'>, 'value_str': <class 'float'>, 'starttime': <class 'numpy.float64'>, 'duration': <class 'numpy.float64'>, 'isValid': <class 'bool'>, 'patient_id': <class 'numpy.int64'>}


In [ ]:
# 4. TODO: merge the summarizations to one summarization file

In [ ]:
# 5. database setup

# Next TODO s: Creeate postgres DB (Done)
# Implement data io to read/write from to db
# filter and insert data into db
# create indices to optimize performance and optimize docker postgres performance

# provide an interface to access the data


In [ ]:
# 5. Filter relevant data based on txt file path selection and insert in postgres db table





In [ ]:
# 6. use csv file or database to access data

In [ ]:
# 7. transform table in pivoted cleaned and accessable table (in postgres with login)

## Playground

Code snipts here are work in progress.

In [ ]:
# 4. merge the summarizations

In [ ]:
#merged_stats = merge_csv_files(stat_file_names)
#merged_stats

### Playgorund - Chat GPT Snippets

In [ ]:
import numpy as np

aggregation = {
    "value_count": "sum", 
    "mean": lambda x: np.sum(x)/np.sum(merged_stats["value_count"]),
    "max": "max",
    "min": "min",
    "mean_duration": "mean"
}

df_aggregated = merged_stats.groupby("path").agg(aggregation)
df_aggregated

#return df_aggregated

In [ ]:

# group by path and aggregate
out = df.groupby('path').agg({'value_count': 'sum', 'mean': lambda x: sum(x)/sum(df['value_count']),
                              'max': 'max', 'min': 'min', 'mean_duration': 'mean'})

In [ ]:
df = pd.read_csv(file_names[0])

In [ ]:
import numpy as np
import pandas as pd

# define list of file names
csv_files = ['file1.csv', 'file2.csv', 'file3.csv', 'file4.csv']

# create empty dataframe
merged_df = pd.DataFrame()

# loop through files
for csv in csv_files:
    # read csv
    df = pd.read_csv(csv)
    
    # calculate aggregates
    df['mean'] = df['mean'] * df['value_count']
    df['min'] = df['min'] * df['value_count']
    df['max'] = df['max'] * df['value_count']
    df['mean_duration'] = df['mean_duration'] * df['value_count']
    
    # group by path
    df_grouped = df.groupby('path').sum()
    
    # calculate new aggregates
    df_grouped['mean'] = df_grouped['mean'] / df_grouped['value_count']
    df_grouped['min'] = df_grouped['min'] / df_grouped['value_count']
    df_grouped['max'] = df_grouped['max'] / df_grouped['value_count']
    df_grouped['mean_duration'] = df_grouped['mean_duration'] / df_grouped['value_count']
    
    # append to merged dataframe
    merged_df = pd.concat([merged_df, df_grouped])
    
# drop duplicates and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)

# display dataframe
print(merged_df)